In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tiktoken
import pandas as pd
import random
from itertools import combinations
import math

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_json('/kaggle/input/afasdfasfasf12313/data_.json')
df



,Resume_str,Resume_html,Category,Category_numberic
0,"{\n ""contact_information"": {\n ""full_name""...","<div class=""fontsize fontface vmargins hmargin...",HR,0
1,"{\n ""contact_information"": {\n ""name"": ""Ja...","<div class=""fontsize fontface vmargins hmargin...",HR,0
2,"{\n ""contactInformation"": {\n ""fullName"": ...","<div class=""fontsize fontface vmargins hmargin...",HR,0
3,"{\n ""personal_information"": {\n ""name"": ""J...","<div class=""fontsize fontface vmargins hmargin...",HR,0
4,"{\n ""contact_information"": {\n ""name"": ""Ja...","<div class=""fontsize fontface vmargins hmargin...",HR,0
...,...,...,...,...
1597,"{\n ""personal_info"": {\n ""name"": ""Unknown""...","<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA,12
1598,lỗi api,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA,12
1599,lỗi api,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA,12
1600,lỗi api,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA,12


In [3]:
# prompt: Using dataframe df: tôi muốn xóa các hàng có cột Resume_str là lỗi api

# Remove rows where 'Resume_str' is 'lỗi api'
df = df[df['Resume_str'] != 'lỗi api']

In [6]:
df.shape

(1591, 4)

In [4]:
# prompt: chia df thành train và validation

from sklearn.model_selection import train_test_split
train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

print("Kích thước tập train:", train_df.shape)
print("Kích thước tập validation:", validation_df.shape)

Kích thước tập train: (1272, 4)
Kích thước tập validation: (319, 4)


In [5]:
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=50, overlap=10):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.overlap = overlap
        # Convert the 'Resume_str' column to string type before processing
        self.titles = dataframe['Resume_str'].astype(str).str.lower().values
        self.labels = dataframe['Category_numberic'].values
        self.chunks = []
        self.chunk_labels = []

        # Chunking với overlap
        for idx, title in enumerate(self.titles):
            # Ensure title is not None or NaN before encoding
            if isinstance(title, str):
                encoding = self.tokenizer.encode(title)
                if len(encoding) <= max_length:
                    self.chunks.append(encoding)
                    self.chunk_labels.append(self.labels[idx])
                else:
                    for start in range(0, len(encoding), max_length - overlap):
                        end = min(start + max_length, len(encoding))
                        chunk = encoding[start:end]
                        self.chunks.append(chunk)
                        self.chunk_labels.append(self.labels[idx])
            else:
                 # Optional: Handle cases where after astype(str) it's still not a valid string (e.g., pandas NaN becoming 'nan')
                 # For this specific case, 'nan' will be tokenized by tiktoken, which is usually acceptable.
                 # If you need to skip non-string like values, add a check here.
                 pass # or print a warning/error


    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        chunk = self.chunks[idx]
        label = self.chunk_labels[idx]
        input_ids = torch.tensor(chunk, dtype=torch.long)
        return input_ids, label

In [6]:
class PairTextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=50, overlap=10, max_pairs=10000):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.overlap = overlap
        self.max_pairs = max_pairs
        # Convert the 'Resume_str' column to string type before processing
        self.titles = dataframe['Resume_str'].astype(str).str.lower().values
        self.labels = dataframe['Category_numberic'].values
        self.chunks = []
        self.chunk_labels = []
        self.pairs = []
        self.pair_labels = []

        # Chunking với overlap
        for idx, title in enumerate(self.titles):
            # Ensure title is not None or NaN before encoding
            if isinstance(title, str):
                encoding = self.tokenizer.encode(title)
                if len(encoding) <= max_length:
                    self.chunks.append(encoding)
                    self.chunk_labels.append(self.labels[idx])
                else:
                    for start in range(0, len(encoding), max_length - overlap):
                        end = min(start + max_length, len(encoding))
                        chunk = encoding[start:end]
                        self.chunks.append(chunk)
                        self.chunk_labels.append(self.labels[idx])
            else:
                 # Optional: Handle cases where after astype(str) it's still not a valid string
                 pass # or print a warning/error


        # Tạo cặp tiêu đề
        indices = list(range(len(self.chunks)))
        random.seed(42)
        same_label_pairs = []
        for label in set(self.chunk_labels):
            label_indices = [i for i, l in enumerate(self.chunk_labels) if l == label]
            same_label_pairs.extend(list(combinations(label_indices, 2)))
        diff_label_pairs = []
        for i in indices:
            for j in indices:
                if self.chunk_labels[i] != self.chunk_labels[j]:
                    diff_label_pairs.append((i, j))

        random.shuffle(same_label_pairs)
        random.shuffle(diff_label_pairs)
        num_same = min(len(same_label_pairs), max_pairs // 2)
        num_diff = min(len(diff_label_pairs), max_pairs // 2)
        self.pairs = same_label_pairs[:num_same] + diff_label_pairs[:num_diff]
        self.pair_labels = [1] * num_same + [0] * num_diff

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        idx1, idx2 = self.pairs[idx]
        chunk1 = self.chunks[idx1]
        chunk2 = self.chunks[idx2]
        label = self.pair_labels[idx]
        input_ids1 = torch.tensor(chunk1, dtype=torch.long)
        input_ids2 = torch.tensor(chunk2, dtype=torch.long)
        return input_ids1, input_ids2, label

In [7]:
def collate_fn_single(batch):
    input_ids = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    max_length = max(len(ids) for ids in input_ids)
    input_ids = torch.stack([
        torch.cat([ids, torch.zeros(max_length - len(ids), dtype=torch.long)])
        for ids in input_ids
    ])
    labels = torch.tensor(labels, dtype=torch.long)
    return input_ids.to(device), labels.to(device)

# Collate function cho bài toán cặp
def collate_fn_pair(batch):
    input_ids1 = [item[0] for item in batch]
    input_ids2 = [item[1] for item in batch]
    labels = [item[2] for item in batch]
    max_length1 = max(len(ids) for ids in input_ids1)
    max_length2 = max(len(ids) for ids in input_ids2)
    max_length = max(max_length1, max_length2)
    input_ids1 = torch.stack([
        torch.cat([ids, torch.zeros(max_length - len(ids), dtype=torch.long)])
        for ids in input_ids1
    ])
    input_ids2 = torch.stack([
        torch.cat([ids, torch.zeros(max_length - len(ids), dtype=torch.long)])
        for ids in input_ids2
    ])
    labels = torch.tensor(labels, dtype=torch.long)
    return input_ids1.to(device), input_ids2.to(device), labels.to(device)

In [8]:

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return x


In [ ]:

# Transformer Encoder Layer
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads
        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)
        self.dense = nn.Linear(d_model, d_model)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model)
        )
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.transpose(1, 2)

    def scaled_dot_product_attention(self, q, k, v, mask=None):
        matmul_qk = torch.matmul(q, k.transpose(-2, -1))
        dk = torch.tensor(k.size(-1), dtype=torch.float32)
        scaled_attention_logits = matmul_qk / torch.sqrt(dk)
        if mask is not None:
            scaled_attention_logits = scaled_attention_logits.masked_fill(mask == 0, -1e9)
        attention_weights = torch.nn.functional.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)
        return output, attention_weights

    def forward(self, x, mask=None):
        batch_size = x.size(0)
        q = self.split_heads(self.wq(x), batch_size)
        k = self.split_heads(self.wk(x), batch_size)
        v = self.split_heads(self.wv(x), batch_size)
        scaled_attention, _ = self.scaled_dot_product_attention(q, k, v, mask)
        scaled_attention = scaled_attention.transpose(1, 2).contiguous()
        concat_attention = scaled_attention.view(batch_size, -1, self.d_model)
        attn_output = self.dense(concat_attention)
        x = self.layernorm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.layernorm2(x + self.dropout(ff_output))
        return x

# Transformer Model (chỉ encoder)
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, embed_size, d_model, num_heads, d_ff, num_layers=4, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(d_model)
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        for layer in self.encoder_layers:
            x = layer(x, mask)
        return x  # (batch_size, seq_len, d_model)

# Classifier cho bài toán đa lớp
class SingleClassifier(nn.Module):
    def __init__(self, transformer_model, d_model, output_size, dropout=0.1):
        super(SingleClassifier, self).__init__()
        self.transformer = transformer_model
        self.fc1 = nn.Linear(d_model, 256, activation=nn.ReLU())
        self.fc2 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x = self.transformer(x, mask)
        x = x.mean(dim=1)
        x = self.fc1(self.dropout(x))
        x = self.fc2(self.dropout(x))
        return x



In [10]:
tokenizer = tiktoken.get_encoding('gpt2')

# Dataset cho bài toán đa lớp
train_dataset_single = TextDataset(train_df, tokenizer, max_length=500, overlap=45)
val_dataset_single = TextDataset(validation_df, tokenizer, max_length=500, overlap=45)

# Dataset cho bài toán cặp
train_dataset_pair = PairTextDataset(train_df, tokenizer, max_length=500, overlap=45, max_pairs=10000)
val_dataset_pair = PairTextDataset(validation_df, tokenizer, max_length=500, overlap=45, max_pairs=2000)






In [11]:
batch_size = 32
train_dataloader_single = DataLoader(train_dataset_single, batch_size=batch_size, shuffle=True, collate_fn=collate_fn_single)
val_dataloader_single = DataLoader(val_dataset_single, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_single)



In [12]:
# Initialize the Transformer model
vocab_size = tokenizer.n_vocab
embed_size = 512
d_model = 512
num_heads = 8
d_ff = 512
num_layers = 4
dropout = 0.1

transformer_model = TransformerModel(vocab_size, embed_size, d_model, num_heads, d_ff, num_layers, dropout)

In [67]:
# Huấn luyện bài toán đa lớp
print("Training Single Classifier (Multi-class)...")
output_size_single = len(train_df['Category_numberic'].unique())
single_model = SingleClassifier(transformer_model, d_model, output_size_single, dropout)
single_model.to(device)

num_epochs = 20
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(single_model.parameters(), lr=0.008)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)
best_val_acc = 0.0
patience = 5
patience_counter = 0

Training Single Classifier (Multi-class)...


In [68]:
for epoch in range(10):
    single_model.train()
    total_loss = 0
    for input_ids, labels in train_dataloader_single:
        optimizer.zero_grad()
        outputs = single_model(input_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader_single)
    print(f"Single Classifier - Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    if epoch > 15:
     # Validation
      single_model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
          for input_ids, labels in val_dataloader_single:
              outputs = single_model(input_ids)
              _, predicted = torch.max(outputs, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

      val_acc = 100 * correct / total
      print(f"Single Classifier - Validation Accuracy after Epoch {epoch+1}: {val_acc:.2f}%")

      scheduler.step(val_acc)
      if val_acc > best_val_acc:
          best_val_acc = val_acc
          patience_counter = 0
          


Single Classifier - Epoch 1/20, Loss: 3.0369
Single Classifier - Epoch 2/20, Loss: 3.0439
Single Classifier - Epoch 3/20, Loss: 3.0266
Single Classifier - Epoch 4/20, Loss: 3.0230
Single Classifier - Epoch 5/20, Loss: 3.0302
Single Classifier - Epoch 6/20, Loss: 3.0293
Single Classifier - Epoch 7/20, Loss: 3.0386
Single Classifier - Epoch 8/20, Loss: 3.0347


KeyboardInterrupt: 

In [16]:
torch.save(single_model.transformer.state_dict(), 'best_transformer_encoder_single.pt')

In [ ]:
# Classifier cho bài toán cặp
class PairClassifier(nn.Module):
    def __init__(self, transformer_model, d_model, output_size=2, dropout=0.1):
        super(PairClassifier, self).__init__()
        self.transformer = transformer_model
        self.fc1 = nn.Linear(d_model, 256, activation=nn.ReLU())
        self.fc2 = nn.Linear(256, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x1, x2, mask=None):
        x1 = self.transformer(x1, mask)
        x1 = x1.mean(dim=1)
        x2 = self.transformer(x2, mask)
        x2 = x2.mean(dim=1)
        x12 = (x1 + x2)/2
        x = self.fc1(self.dropout(x12))
        x = self.fc2(self.dropout(x))
        return x

In [ ]:
train_dataloader_pair = DataLoader(train_dataset_pair, batch_size=16, shuffle=True, collate_fn=collate_fn_pair)
val_dataloader_pair = DataLoader(val_dataset_pair, batch_size=16, shuffle=False, collate_fn=collate_fn_pair)

In [ ]:
print("\nTraining Pair Classifier (Binary)...")
pair_model = PairClassifier(single_model.transformer, d_model, output_size=2, dropout=dropout)
pair_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pair_model.parameters(), lr=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)
best_val_acc = 0.0
patience_counter = 0

for epoch in range(1):
    pair_model.train()
    total_loss = 0
    for input_ids1, input_ids2, labels in train_dataloader_pair:
        optimizer.zero_grad()
        outputs = pair_model(input_ids1, input_ids2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader_pair)
    print(f"Pair Classifier - Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    # Validation
    pair_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for input_ids1, input_ids2, labels in val_dataloader_pair:
            outputs = pair_model(input_ids1, input_ids2)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    print(f"Pair Classifier - Validation Accuracy after Epoch {epoch+1}: {val_acc:.2f}%")

    scheduler.step(val_acc)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(pair_model.transformer.state_dict(), 'best_transformer_encoder_pair.pt')



In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
import tiktoken

# Lớp Dataset cho MLM
class MLMDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=500, mask_prob=0.15, random_token_prob=0.1):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_prob = mask_prob  # Xác suất che token
        self.random_token_prob = random_token_prob  # Xác suất thay bằng token ngẫu nhiên
        self.vocab_size = tokenizer.n_vocab
        # Convert Resume_str to string and lowercase
        self.titles = dataframe['Resume_str'].astype(str).str.lower().values

        self.chunks = []
        for title in self.titles:
            if isinstance(title, str):
                encoding = self.tokenizer.encode(title)
                # Chia thành các chunk nếu vượt quá max_length
                for start in range(0, len(encoding), max_length):
                    end = min(start + max_length, len(encoding))
                    chunk = encoding[start:end]
                    self.chunks.append(chunk)

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        chunk = self.chunks[idx]
        input_ids = torch.tensor(chunk, dtype=torch.long)
        labels = input_ids.clone()  # Nhãn là token gốc

        # Tạo mask ngẫu nhiên
        rand = torch.rand(input_ids.shape)
        mask_arr = (rand < self.mask_prob) * (input_ids != self.tokenizer.encode('[PAD]')[0])  # Không mask padding

        # Tạo các loại mask: [MASK] token, token ngẫu nhiên, hoặc giữ nguyên
        selection = []
        for i in range(input_ids.shape[0]):
            if mask_arr[i]:
                if random.random() < 0.8:  # 80% thay bằng [MASK]
                    input_ids[i] = self.tokenizer.encode('[MASK]')[0]
                elif random.random() < 0.5:  # 10% thay bằng token ngẫu nhiên
                    input_ids[i] = random.randint(0, self.vocab_size - 1)
                # 10% giữ nguyên token gốc
            selection.append(mask_arr[i])

        # Chỉ tính loss trên các vị trí được mask
        labels[~mask_arr] = -100  # CrossEntropyLoss bỏ qua nhãn -100

        return input_ids, labels

# Mô hình cho MLM
class MLMClassifier(nn.Module):
    def __init__(self, transformer_model, d_model, vocab_size):
        super(MLMClassifier, self).__init__()
        self.transformer = transformer_model
        self.fc = nn.Linear(d_model, vocab_size)  # Dự đoán token trong từ vựng

    def forward(self, x, mask=None):
        x = self.transformer(x, mask)
        x = self.fc(x)  # Dự đoán cho mỗi token
        return x

# Hàm collate cho MLM
def collate_fn_mlm(batch):
    input_ids = [item[0] for item in batch]
    labels = [item[1] for item in batch]
    max_length = max(len(ids) for ids in input_ids)
    input_ids = torch.stack([
        torch.cat([ids, torch.zeros(max_length - len(ids), dtype=torch.long)])
        for ids in input_ids
    ])
    labels = torch.stack([
        torch.cat([lbl, torch.full((max_length - len(lbl),), -100, dtype=torch.long)])
        for lbl in labels
    ])
    return input_ids.to(device), labels.to(device)

# Khởi tạo dataset và dataloader
def prepare_mlm_data(train_df, validation_df, tokenizer, batch_size=16):
    train_dataset_mlm = MLMDataset(train_df, tokenizer, max_length=500)
    val_dataset_mlm = MLMDataset(validation_df, tokenizer, max_length=500)

    train_dataloader_mlm = DataLoader(
        train_dataset_mlm,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn_mlm
    )
    val_dataloader_mlm = DataLoader(
        val_dataset_mlm,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn_mlm
    )

    return train_dataloader_mlm, val_dataloader_mlm

# Hàm huấn luyện MLM
def train_mlm(model, train_dataloader, val_dataloader, device, num_epochs=20):
    model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)  # Bỏ qua nhãn -100
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=3, verbose=True
    )
    best_val_loss = float('inf')
    patience_counter = 0
    patience = 5

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for input_ids, labels in train_dataloader:
            optimizer.zero_grad()
            outputs = model(input_ids)
            # Reshape outputs và labels cho CrossEntropyLoss
            outputs = outputs.view(-1, outputs.size(-1))
            labels = labels.view(-1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        print(f"MLM - Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")
        if epoch > 15:
          # Validation
          model.eval()
          correct = 0.0
          total_val_loss = 0
          with torch.no_grad():
              for input_ids, labels in val_dataloader:
                  outputs = pair_model(input_ids)
                  _, predicted = torch.max(outputs, 1)
                  total += labels.size(0)
                  correct += (predicted == labels).sum().item()
          val_acc = 100 * correct / total
          print(f"Pair Classifier - Validation Accuracy after Epoch {epoch+1}: {val_acc:.2f}%")

          



In [32]:
train_dataloader_mlm, val_dataloader_mlm = prepare_mlm_data(train_df, validation_df, tokenizer)



In [ ]:
    # Khởi tạo mô hình MLM
vocab_size = tokenizer.n_vocab
d_model = 512  # Phải khớp với TransformerModel


In [ ]:
mlm_model = MLMClassifier(pair_model.transformer, d_model, vocab_size)



In [ ]:
    # Huấn luyện
train_mlm(mlm_model, train_dataloader_mlm, val_dataloader_mlm, device)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


MLM - Epoch 1/20, Train Loss: 5.4661
MLM - Epoch 2/20, Train Loss: 5.0845


KeyboardInterrupt: 

In [17]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Định nghĩa lại TransformerModel (từ notebook của bạn)


# Hàm inference để tạo embedding
def get_embedding(text, tokenizer, model, max_length=500, device='cuda'):
    # Chuyển model sang chế độ đánh giá
    model.eval()
    
    # Tiền xử lý văn bản
    text = str(text).lower()  # Chuyển thành lowercase
    encoding = tokenizer.encode(text)
    
    # Chia thành các chunk nếu dài hơn max_length
    chunks = []
    for start in range(0, len(encoding), max_length):
        end = min(start + max_length, len(encoding))
        chunk = encoding[start:end]
        chunks.append(chunk)
    # print(len(chunks))
    
    # Tạo embedding cho từng chunk
    embeddings = []
    with torch.no_grad():
        for chunk in chunks:
            input_ids = torch.tensor([chunk], dtype=torch.long).to(device)  # Thêm batch dimension
            output = model(input_ids)  # (1, seq_len, d_model)
            # Mean pooling để lấy embedding cố định cho chunk
            embedding = output.mean(dim=1).squeeze().cpu().numpy()  # (d_model,)
            embeddings.append(embedding)
    
    # Nếu có nhiều chunk, lấy trung bình các embedding
    if len(embeddings) > 1:
        embedding = np.mean(embeddings, axis=0)
    else:
        embedding = embeddings[0]
    
    return embedding



In [93]:
len(df)

1591

In [43]:
# Khởi tạo tokenizer
tokenizer = tiktoken.get_encoding('gpt2')

# Khởi tạo mô hình
vocab_size = tokenizer.n_vocab
embed_size = 512
d_model = 512
num_heads = 8
d_ff = 512
num_layers = 4
dropout = 0.1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TransformerModel(vocab_size, embed_size, d_model, num_heads, d_ff, num_layers, dropout)
model.to(device)

# Tải trọng số mô hình đã huấn luyện
model.load_state_dict(torch.load('best_transformer_encoder_single.pt', map_location=device))

# Văn bản mẫu để tạo embedding
sample_text = '''

}
'''

# Tạo embedding
embedding = get_embedding(sample_text, tokenizer, model, max_length=500, device=device)
print("Embedding shape:", embedding.shape)
print("Embedding:", embedding)

Embedding shape: (512,)
Embedding: [-5.02282614e-03  2.34679244e-02 -2.40494050e-02  5.16255945e-02
  1.46794794e-02 -4.66561168e-02  3.85042652e-02  6.61958940e-03
 -2.06532888e-02  1.86391489e-03 -9.44417063e-03 -2.86207683e-02
  4.09292951e-02 -5.18304436e-03 -1.08358534e-02  1.71186379e-03
 -2.55484283e-02 -1.55053390e-02 -1.43661788e-02 -2.64415033e-02
  1.51722301e-02 -3.20907533e-02 -2.90663145e-03  1.28827263e-02
 -3.82271074e-02  5.76426424e-02 -8.11788999e-03  2.07635649e-02
  2.24078111e-02  1.44596286e-02  1.95403006e-02  1.31088179e-02
  7.80927250e-04  1.68923698e-02  3.46968393e-03 -4.00016569e-02
 -4.18220833e-02 -1.21923136e-02 -9.65999346e-03 -2.11737715e-02
 -3.54035338e-03  9.43067949e-03 -9.71361995e-03  1.38877034e-02
 -3.59541215e-02 -1.47311417e-02 -2.14921799e-03 -1.90584157e-02
 -7.23283365e-03 -3.35974507e-02 -2.08429806e-02 -9.29169636e-03
 -4.49880864e-03 -2.32714396e-02 -1.47337755e-02 -1.52520193e-02
  2.44010065e-04  6.32641139e-03 -1.48765482e-02  5.398

In [21]:
from langchain.vectorstores import FAISS
import numpy as np
from langchain.embeddings import FakeEmbeddings  # placeholder nếu không dùng LangChain embedding model

# embeddings là list[np.ndarray]
embedding_dim = embeddings[0].shape[0]
embeddings_np = np.array(embeddings).astype("float32")

# FAISS yêu cầu dùng FAISS-compatible embeddings interface từ langchain
class CustomEmbedding:
    def embed_documents(self, texts):
        return embeddings_np.tolist()

    def embed_query(self, text):
        return get_embedding(text, tokenizer, model, max_length=500, device=device).tolist()

vectorstore = FAISS.from_documents(documents, embedding=CustomEmbedding())


NameError: name 'embeddings' is not defined

In [22]:
import json
with open ('/kaggle/input/afasdfasfasf12313/data_.json','r') as f:
    data = json.load(f)

print(len(data))
    

1602


In [36]:
import pandas as pd
df = pd.read_json('/kaggle/input/afasdfasfasf12313/data_.json')

In [79]:
df = df[df['Resume_str']!='lỗi api']

In [54]:
data[0].keys()

dict_keys(['Resume_str', 'Resume_html', 'Category', 'Category_numberic', 'embedding'])

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts = []
for d in data:
    text = d.get("Resume_str")
    if text is not None:
        texts.append(text.lower())
    else:
        texts.append("abcxyz")  # hoặc bỏ qua, hoặc cho giá trị mặc định

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(texts).toarray()


In [38]:
new_data =[]
for i in range(len(data)):
    if data[i]['Resume_str'] is not None:
        new_data.append(data[i])
len(new_data)

1598

In [41]:
new_data[0]

{'Resume_str': '{\n  "contact_information": {\n    "full_name": "Jane Doe",\n    "phone_number": "123-456-7890",\n    "email": "jane.doe@example.com"\n  },\n  "profile": {\n    "title": "HR Administrator / Marketing Associate",\n    "summary": "Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management. Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.",\n    "highlights": [\n      "Focused on customer satisfaction",\n      "Team management",\n      "Marketing savvy",\n      "Conflict resolution techniques",\n      "Training and development",\n      "Skilled multi-tasker",\n      "Client relations specialist"\n    ],\n    "accomplishments": [\n      "Missouri DOT Supervisor Training Certification",\n      "Certified by IHG in Customer Loyalty and Marketing by Segment",\n      "Hilton Worldwide General Manager Training Certification",\n      "Accomplished T

In [50]:
for i in range(len(new_data)):
    new_data[i]['embedding'] = get_embedding(new_data[i]['Resume_str'], tokenizer, model, max_length=500, device=device).tolist()
len(new_data)

1598

In [52]:
new_data[1]['embedding']

[-0.005022851284593344,
 0.023467902094125748,
 -0.024049388244748116,
 0.05162565037608147,
 0.014679468236863613,
 -0.0466562882065773,
 0.03850414976477623,
 0.006619584746658802,
 -0.02065330184996128,
 0.0018639098852872849,
 -0.009444170631468296,
 -0.028620826080441475,
 0.04092959687113762,
 -0.005183029919862747,
 -0.01083581056445837,
 0.001711921882815659,
 -0.02554837055504322,
 -0.015505223535001278,
 -0.014366189017891884,
 -0.02644166350364685,
 0.01517221424728632,
 -0.03209079056978226,
 -0.002906643319875002,
 0.012882719747722149,
 -0.03822706267237663,
 0.05764254555106163,
 -0.008117885328829288,
 0.020763562992215157,
 0.022407809272408485,
 0.014459580183029175,
 0.01954035274684429,
 0.013108850456774235,
 0.0007807841175235808,
 0.016892390325665474,
 0.003469681367278099,
 -0.04000183939933777,
 -0.041822079569101334,
 -0.012192336842417717,
 -0.009659885428845882,
 -0.021173836663365364,
 -0.0035403710789978504,
 0.009430722333490849,
 -0.00971348024904728,
 

In [53]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

text_embedding_pairs = [(d["Resume_html"], d["embedding"]) for d in new_data]
faiss_index = FAISS.from_embeddings(text_embedding_pairs)



TypeError: FAISS.from_embeddings() missing 1 required positional argument: 'embedding'

In [102]:
ffaiss_index.save_local("faiss_with_tfidf")


In [64]:
from langchain.vectorstores import FAISS
from langchain.schema import Document
import torch
import numpy as np

# Load lại FAISS từ file local (đường dẫn folder chứa index và metadata)
faiss_index = FAISS.load_local(
    folder_path="/kaggle/working/faiss_index",
    embeddings=None,
    allow_dangerous_deserialization=True  # Bắt buộc phải thêm tham số này
)

# Ví dụ query text
query_text = """         HR ADMINISTRATOR/MARKETING ASSOCIATE

HR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss prevention, safety, time management, leadership and performance assessment.        Experience      HR Administrator/Marketing Associate

HR Administrator     Dec 2013   to   Current      Company Name   ï¼   City  ,   State     Helps to develop policies, directs and coordinates activities such as employment, compensation, labor relations, benefits, training, and employee services.  Prepares employee separation notices and related documentation  Keeps records of benefits plans participation such as insurance and pension plan, personnel transactions such as hires, promotions, transfers, performance reviews, and terminations, and employee statistics for government reporting.  Advises management in appropriate resolution of employee relations issues.  Administers benefits programs such as life, health, dental, insurance, pension plans, vacation, sick leave, leave of absence, and employee assistance.     Marketing Associate Â    Designed and created marketing collateral for sales meetings, trade shows and company executives.  Managed the in-house advertising program consisting of print and media collateral pieces.  Assisted in the complete design and launch of the company's website in 2 months.  Created an official company page on Facebook to facilitate interaction with customers.  Analyzed ratings and programming features of competitors to evaluate the effectiveness of marketing strategies.         Advanced Medical Claims Analyst     Mar 2012   to   Dec 2013      Company Name   ï¼   City  ,   State     Reviewed medical bills for the accuracy of the treatments, tests, and hospital stays prior to sanctioning the claims.  Trained to interpret the codes (ICD-9, CPT) and terminology commonly used in medical billing to fully understand the paperwork that is submitted by healthcare providers.  Required to have organizational and analytical skills as well as computer skills, knowledge of medical terminology and procedures, statistics, billing standards, data analysis and laws regarding medical billing.         Assistant General Manager     Jun 2010   to   Dec 2010      Company Name   ï¼   City  ,   State     Performed duties including but not limited to, budgeting and financial management, accounting, human resources, payroll and purchasing.  Established and maintained close working relationships with all departments of the hotel to ensure maximum operation, productivity, morale and guest service.  Handled daily operations and reported directly to the corporate office.  Hired and trained staff on overall objectives and goals with an emphasis on high customer service.  Marketing and Advertising, working on public relations with the media, government and local businesses and Chamber of Commerce.         Executive Support / Marketing Assistant     Jul 2007   to   Jun 2010      Company Name   ï¼   City  ,   State     Provided assistance to various department heads - Executive, Marketing, Customer Service, Human Resources.  Managed front-end operations to ensure friendly and efficient transactions.  Ensured the swift resolution of customer issues to preserve customer loyalty while complying with company policies.  Exemplified the second-to-none customer service delivery in all interactions with customers and potential clients.         Reservation & Front Office Manager     Jun 2004   to   Jul 2007      Company Name   ï¼   City  ,   State          Owner/ Partner     Dec 2001   to   May 2004      Company Name   ï¼   City  ,   State          Price Integrity Coordinator     Aug 1999   to   Dec 2001      Company Name   ï¼   City  ,   State          Education      N/A  ,   Business Administration   1999     Jefferson College   ï¼   City  ,   State       Business Administration  Marketing / Advertising         High School Diploma  ,   College Prep. studies   1998     Sainte Genevieve Senior High   ï¼   City  ,   State       Awarded American Shrubel Leadership Scholarship to Jefferson College         Skills     Accounting, ads, advertising, analytical skills, benefits, billing, budgeting, clients, Customer Service, data analysis, delivery, documentation, employee relations, financial management, government relations, Human Resources, insurance, labor relations, layout, Marketing, marketing collateral, medical billing, medical terminology, office, organizational, payroll, performance reviews, personnel, policies, posters, presentations, public relations, purchasing, reporting, statistics, website.    
"""

# Lấy embedding query
query_embedding = get_embedding(query_text, tokenizer, model, max_length=500, device=device)

# Tìm kiếm top k kết quả gần nhất trong FAISS
k = 5
results = faiss_index.similarity_search_by_vector(query_embedding, k=k)

# Hiển thị kết quả
for i, doc in enumerate(results):
    print(f"Result {i+1}:")
    print("Text:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("-----")



Result 1:
Text: <div class="fontsize fontface vmargins hmargins linespacing pagesize" id="document"> <div class="section firstsection" id="SECTION_NAME836013982" style="
      padding-top:0px;
    "> <div class="paragraph PARAGRAPH_NAME firstparagraph" id="PARAGRAPH_836013982_1_544840583" style="
      padding-top:0px;
    "> <div class="name" itemprop="name"> <span class="field fName" id="836013982FNAM1"> </span> <span> </span> <span class="field" id="836013982LNAM1"> GUEST TEACHER</span> </div> </div> </div> <div class="section" id="SECTION_SUMM836013987" style="
      padding-top:0px;
    "> <div class="heading bottomborder"> <div class="sectiontitle" id="SECTNAME_SUMM836013987"> Professional Overview</div> </div> <div class="paragraph firstparagraph" id="PARAGRAPH_836013987_1_544840591" style="
      padding-top:0px;
    "> <div class="field singlecolumn" id="836013987FRFM1"> A detail - oriented, high energy Social Science Teacher with the ability to motivate students to work at op

In [61]:
from langchain.vectorstores import FAISS
from langchain.embeddings.base import Embeddings

class DummyEmbedding(Embeddings):
    def embed_documents(self, texts):
        raise NotImplementedError("Not used for indexing — embeddings are already provided.")
    
    def embed_query(self, text):
        get_embedding(text, tokenizer, model, max_length=500, device=device)

dummy_embed = DummyEmbedding()

# Tạo FAISS index
text_embedding_pairs = [(d["Resume_html"], d["embedding"]) for d in new_data]
faiss_index = FAISS.from_embeddings(text_embedding_pairs, embedding=dummy_embed)

In [62]:
faiss_index.save_local("faiss_index")


In [56]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.5 MB/s eta 0:00:00:00:0100:01
